In [1]:
1+1

2

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, ToolMessage
from langgraph.prebuilt import create_react_agent
from typing import TypedDict, Annotated, Sequence
import operator
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

# --- 환경 변수 설정 ---
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")


# --- 기존 도구 함수들 (변경 없음) ---
def add_two(x: str) -> str:
    """숫자 입력 시 2를 더합니다."""
    try:
        return str(float(x) + 2)
    except Exception:
        raise ValueError("숫자만 입력해 주세요.")


def population_lookup(city: str) -> str:
    """도시 이름 입력 시 인구 정보를 반환합니다."""
    data = {
        "서울": "9,733,509명 (2025년 기준)",
        "부산": "3,400,000명 (2025년 기준)",
    }
    # 도시 이름의 양쪽 공백을 제거하고 소문자로 변환하여 일관성 유지
    result = data.get(city.strip().lower())
    if result:
        return result
    else:
        raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# --- 도구 및 LLM 설정 (변경 없음) ---
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 이름 입력 시 인구 정보를 반환합니다. 도시 이름은 '서울', '부산'만 가능합니다.",
    ),
]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPEN_API_KEY)


# --- LangGraph 상태 정의 ---
class AgentState(TypedDict):
    """
    Agent의 상태를 나타냅니다. 메시지 목록을 포함하며, 각 단계에서 메시지가 추가됩니다.

    Attributes:
        messages (Sequence[BaseMessage]): HumanMessage, AIMessage, ToolMessage 등 대화 기록
    """

    messages: Annotated[Sequence[BaseMessage], operator.add]


# --- LangGraph 기반 ReAct Agent 생성 ---
# LangGraph에 사전 빌드된 create_react_agent를 사용하여 Agent 노드를 간단히 생성합니다.
# 이 노드는 내부적으로 ReAct 프롬프트, LLM, 도구를 사용하여 다음 단계를 결정합니다.
react_agent = create_react_agent(llm, tools)


# --- Fallback 함수 정의 ---
# 도구로 해결할 수 없는 경우 호출될 함수
def create_llm_fallback():
    def llm_fallback(state: AgentState) -> dict:
        """
        초기 사용자 질문에 대해 LLM이 직접 답변을 생성하는 Fallback 로직
        """
        # 가장 처음의 사용자 질문을 가져옵니다.
        user_input = state["messages"][0].content

        # LLM을 직접 호출하여 답변 생성
        prompt = f"다음 질문에 답해주세요: {user_input}"
        response = llm.invoke(prompt)

        # Fallback이 실행되었음을 알리는 메시지와 함께 결과 반환
        fallback_message = f"🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n{response.content}"

        return {"messages": [AIMessage(content=fallback_message)]}

    return llm_fallback


# --- Graph 생성 및 컴파일 ---
# StateGraph에 AgentState를 정의하여 그래프를 초기화합니다.
graph = StateGraph(AgentState)

# 노드를 그래프에 추가합니다. 'agent'는 추론, 'action'은 도구 실행을 담당합니다.
# `create_react_agent`는 Agent 로직과 Tool 실행 로직을 합친 노드를 생성합니다.
graph.add_node("agent", react_agent)

# 그래프의 시작점을 'agent' 노드로 설정합니다.
graph.set_entry_point("agent")

# 그래프의 끝을 나타내는 특별한 노드 이름인 END를 사용합니다.
# 'agent' 노드에서 도구 호출이 필요 없으면(즉, 최종 답변이 생성되면) 그래프를 종료합니다.
graph.add_edge("agent", END)

# Fallback 함수를 생성합니다.
llm_fallback_runnable = RunnableLambda(create_llm_fallback())

# 그래프를 컴파일하고, fallback 로직을 연결합니다.
# agent(내부적으로 tool 포함) 실행 중 오류 발생 시 llm_fallback_runnable이 실행됩니다.
smart_agent = graph.compile().with_fallbacks(fallbacks=[llm_fallback_runnable])

# --- 테스트 ---
if __name__ == "__main__":
    print("=== 도구에서 찾을 수 있는 정보 ===")
    # 초기 상태를 HumanMessage와 함께 딕셔너리 형태로 제공합니다.
    inputs1 = {"messages": [HumanMessage(content="서울의 인구는?")]}
    result1 = smart_agent.invoke(inputs1)
    # 최종 결과는 'messages' 리스트의 마지막 AIMessage에 담겨 있습니다.
    print(result1["messages"][-1].content)

    print("\n" + "=" * 40 + "\n")

    print("=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===")
    inputs2 = {"messages": [HumanMessage(content="대구의 인구는?")]}
    result2 = smart_agent.invoke(inputs2)
    print(result2["messages"][-1].content)

=== 도구에서 찾을 수 있는 정보 ===
서울의 인구는 9,733,509명입니다 (2025년 기준).


=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===
대구의 인구 정보를 제공할 수 없습니다. 서울이나 부산의 인구 정보를 요청해 주시면 도와드리겠습니다.
